In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch

In [2]:
dataset = load_dataset("glue", "qqp")
test_data = dataset["validation"].shuffle(seed=40).select(range(1500))

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
model_name = "dog-in-the-box-studio/qpp-roberta-lora"  # Replace with your fine-tuned model directory if saved locally
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum ModelWrapper at line 250370 column 3

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["question1"],
        examples["question2"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
tokenized_test_data = test_data.map(preprocess_function, batched=True)

In [ ]:
def prepare_data(data):
    input_ids = torch.tensor(data["input_ids"]).to(device)
    attention_mask = torch.tensor(data["attention_mask"]).to(device)
    labels = torch.tensor(data["label"]).to(device)
    return input_ids, attention_mask, labels

test_input_ids, test_attention_mask, test_labels = prepare_data(tokenized_test_data)

In [ ]:
def evaluate_model(model, input_ids, attention_mask, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()  # Move predictions back to CPU for evaluation
        true_labels = labels.cpu().numpy()  # Move true labels back to CPU for evaluation
        return predictions, true_labels

predictions, true_labels = evaluate_model(model, test_input_ids, test_attention_mask, test_labels)


In [ ]:
tp = 0  # True Positives
fp = 0  # False Positives
tn = 0  # True Negatives
fn = 0  # False Negatives

# Compute confusion matrix values
for pred, label in zip(predictions, true_labels):
    if pred == 1 and label == 1:
        tp += 1
    elif pred == 1 and label == 0:
        fp += 1
    elif pred == 0 and label == 0:
        tn += 1
    elif pred == 0 and label == 1:
        fn += 1

# Accuracy
accuracy = (tp + tn) / (tp + fp + tn + fn)

# Precision
precision = tp / (tp + fp) if (tp + fp) > 0 else 0

# Recall
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

# F1-Score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Step 7: Print Results
print(f"Accuracy: {accuracy*100}")
print(f"Precision: {precision*100}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1*100}")
